In [1]:
import multiprocessing, geneffect, firm

/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/Bio/Align/substitution_matrices/__init__.py:21: BiopythonExperimentalWarning: Bio.Align.substitution_matrices is an experimental module which may still undergo significant changes. In particular, the location of this module may change, and the Array class defined in this module may be moved to other existing or new modules in Biopython.
  BiopythonExperimentalWarning)


In [2]:
n_threads = 4
thread_pool = multiprocessing.Pool(n_threads)
geneffect_setup = geneffect.Setup('GRCh38') # Must be a human reference genome
firm.setup_uniprot_tracks(geneffect_setup)
firm_classifier = firm.load_classifier(geneffect_setup)


GENEFFECT|PID-30865 [2021-08-16 17:50:46.184494]: Working with reference genome version GRCh38 (user specified: GRCh38).
GENEFFECT|PID-30865 [2021-08-16 17:50:48.509691]: Loaded 93748 pfam records, 51710 of which are domains. Found 347 unique clans within these domains, 173 of which are common (with at least 25 occurrences).
GENEFFECT|PID-30865 [2021-08-16 17:52:24.224633]: Parsed 20395 UniProt records.
GENEFFECT|PID-30865 [2021-08-16 17:52:58.982540]: Loaded 2604486 gene annotations.
GENEFFECT|PID-30865 [2021-08-16 17:52:59.892339]: Loaded 20302 gene meta data records.
GENEFFECT|PID-30865 [2021-08-16 17:53:02.384978]: Constructed 706199 CDS records.


/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/Bio/Seq.py:2748: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


GENEFFECT|PID-30865 [2021-08-16 17:59:53.493533]: Parsed 18641 genes.
GENEFFECT|PID-30865 [2021-08-16 18:00:01.810725]: Finished setting up reference genome GRCh38.


/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/scikit_learn-0.24.1-py3.7-linux-x86_64.egg/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.0 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/scikit_learn-0.24.1-py3.7-linux-x86_64.egg/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.0 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/scikit_learn-0.24.1-py3.7-linux-x86_64.egg/sklearn/base.py:315: UserWarning: Trying to unpickle estimator VarianceThreshold from version 0.24.0 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


ML|PID-30987 [2021-08-16 18:30:27.514716]: Extracting features from [0]...
ML|PID-30989 [2021-08-16 18:30:27.517321]: Extracting features from [1024]...ML|PID-30990 [2021-08-16 18:30:27.517696]: Extracting features from [1536]...ML|PID-30988 [2021-08-16 18:30:27.518526]: Extracting features from [512]...




In [4]:
snp = geneffect_setup.variant_interpreter.process_snp('17', 43082434, 'G', 'C')
snp_gene_effect, = snp.cds_gene_effects
print(snp_gene_effect)
print(firm_classifier.predict_adjusted_proba(snp_gene_effect))

P38398:R1443G
0.09052460784481486


In [5]:
async_firm_classifier = firm.AsyncClassifier(firm_classifier.predict_adjusted_proba, thread_pool = thread_pool, n_threads = n_threads)
async_firm_classifier.submit_samples(2000 * [snp_gene_effect], callback = lambda scores: print(len(scores), scores[:10]))
async_firm_classifier.process_remaining_samples()
async_firm_classifier.close()


ML|PID-30865 [2021-08-16 18:30:27.432768]: Submitted 2000 new samples to classify. There are now 2000 pending samples.
ML|PID-30865 [2021-08-16 18:30:27.434013]: Classifying a batch of 2000 samples...
2000 [0.09052460784481482, 0.09052460784481482, 0.09052460784481482, 0.09052460784481482, 0.09052460784481482, 0.09052460784481482, 0.09052460784481482, 0.09052460784481482, 0.09052460784481482, 0.09052460784481482]


In [16]:
# firm_classifier.feature_extractor.create_features_as_dict(snp_gene_effect)
# write code that selects a changed vriant and firm effect score, have me and nadav run it
dict(zip(firm_classifier.feature_extractor.get_feature_names(),firm_classifier.feature_extractor.create_features_as_array(snp_gene_effect)))

{'protein_length': 1863.0,
 'variant_position': 1443.0,
 'variant_relative_position': 0.7745571658615137,
 'context_global_aa_A': 84.0,
 'context_global_aa_C': 44.0,
 'context_global_aa_D': 85.0,
 'context_global_aa_E': 198.0,
 'context_global_aa_F': 49.0,
 'context_global_aa_G': 87.0,
 'context_global_aa_H': 49.0,
 'context_global_aa_I': 77.0,
 'context_global_aa_K': 137.0,
 'context_global_aa_L': 156.0,
 'context_global_aa_M': 30.0,
 'context_global_aa_N': 121.0,
 'context_global_aa_P': 96.0,
 'context_global_aa_Q': 97.0,
 'context_global_aa_S': 224.0,
 'context_global_aa_R': 76.0,
 'context_global_aa_T': 111.0,
 'context_global_aa_V': 101.0,
 'context_global_aa_W': 10.0,
 'context_global_aa_Y': 31.0,
 'context_global_aa__': 0.0,
 'context_global_scale_gravy_avg': -0.7849168008588299,
 'context_global_scale_gravy_std': 2.9547550913879026,
 'context_global_scale_ss_helix_avg': 0.22758990874932905,
 'context_global_scale_ss_helix_std': 0.419276450787307,
 'context_global_scale_ordered_